Just a temporary notebook to run the k=15-19 scRNA experiments with the new A.

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0], "code"))

In [7]:
import pytorch_models as models
import pytorch_samplers as samplers
import pytorch_losses as losses
import pytorch_training as training
import pytorch_utils as utils
import sinkhorn_cnf as cnf
import scrna_exper as scrna
from geomloss import SamplesLoss
from torchdiffeq import odeint as odeint

import importlib
import time
import json
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import torch
import torch.nn as nn
import random
import numpy as np

importlib.reload(models)
importlib.reload(losses)
importlib.reload(samplers)
importlib.reload(training)
importlib.reload(utils)
importlib.reload(scrna)
importlib.reload(cnf)

<module 'sinkhorn_cnf' from '/mnt/hdd/scarv/riemannian-metric-learning-ot/code/sinkhorn_cnf.py'>

In [3]:
torch.set_default_dtype(torch.float64)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [4]:
# Import WOT data

data_file = "data/scrna/schiebinger.npz"
data_dict = np.load(data_file, allow_pickle=True)

all_data = data_dict["original_embedding"]
# Rescale the data by factor of 1e-3
all_data *= 1e-3

timestamps = data_dict["sample_labels"]

times = np.unique(timestamps)

subsampled_data_list = []
data_list = []
# Generate training data by randomly selecting 500 observations per time point
for t in list(times):
    data_t = all_data[timestamps == t]
    data_list.append(data_t)
    np.random.shuffle(data_t) # shuffle rows of data_t
    subsampled_data_t = data_t[:500] # keep only first 500 samples
    subsampled_data_list.append(subsampled_data_t)

In [5]:
# Load pretrained model

space_dims = 2
matrix_hidden_dims = 2048
A_fname = "trained_models/scrna_learned_params_v3.pt" # was 'scrna_pretrained_params.pt'
scrna_learnedA = models.PSDMatrix(space_dims, matrix_hidden_dims).to(device)
scrna_learnedA.load_state_dict(torch.load(A_fname))

<All keys matched successfully>

In [6]:
# Generate list of samplers for all time points

rho_list = samplers.generate_all_eb_samplers(data_list)

In [9]:
# Run experiments with A

k_vals = [19]
A_model = scrna_learnedA
for k in k_vals:
    json_fname = "results/scrna_experiments/k" + str(k) + "_withAv2_lambd_1em1.json"
    use_A = True
    W1_vals = scrna.run_experiment(rho_list, json_fname, k, A_model, use_A)
    json_fname_final_tps = "results/scrna_experiments/k" + str(k) + "_withAv2_lambd_1em1_final_tps.json"
    W1_vals_final_tps = scrna.run_final_tp_experiment(rho_list, json_fname_final_tps, k, A_model, use_A)

iter 0 loss: tensor(102.6561, device='cuda:1', grad_fn=<AddBackward0>)
iter 100 loss: tensor(70.7911, device='cuda:1', grad_fn=<AddBackward0>)
iter 200 loss: tensor(67.1092, device='cuda:1', grad_fn=<AddBackward0>)
iter 300 loss: tensor(61.8781, device='cuda:1', grad_fn=<AddBackward0>)
iter 400 loss: tensor(61.3156, device='cuda:1', grad_fn=<AddBackward0>)
iter 500 loss: tensor(58.5950, device='cuda:1', grad_fn=<AddBackward0>)
iter 600 loss: tensor(48.6366, device='cuda:1', grad_fn=<AddBackward0>)
iter 700 loss: tensor(2.1899, device='cuda:1', grad_fn=<AddBackward0>)
iter 800 loss: tensor(2.1129, device='cuda:1', grad_fn=<AddBackward0>)
iter 900 loss: tensor(2.0990, device='cuda:1', grad_fn=<AddBackward0>)
iter 1000 loss: tensor(2.1875, device='cuda:1', grad_fn=<AddBackward0>)
iter 1100 loss: tensor(1.6906, device='cuda:1', grad_fn=<AddBackward0>)
iter 1200 loss: tensor(1.9296, device='cuda:1', grad_fn=<AddBackward0>)
iter 1300 loss: tensor(1.9720, device='cuda:1', grad_fn=<AddBackward

KeyboardInterrupt: 